# Introduction to Big Data Modern Technologies course

## TOPIC 2: NoSQL databases practice - MongoDB
### Part 1

### 1. Libraries

In [ ]:
import os
import sys
import json
import pymongo

Python library [PyMongo](https://pymongo.readthedocs.io) is used to get access to MongoDB demo database. The first step when working with PyMongo is to create a MongoClient to the running mongod instance:

In [ ]:
from pymongo import MongoClient

client = MongoClient() # default setting
client

### 2. MongoClient

In [ ]:
client.server_info()

In [ ]:
client.HOST

In [ ]:
client.nodes

In [ ]:
client.list_database_names()

### 3. Databases and collections

In [ ]:
db = client['testdb']
db

A collection is a group of documents stored in MongoDB, and can be thought of as roughly the equivalent of a table in a relational database. Here is an example how to get a collection:

In [ ]:
collection = db['test_collection']
collection

In [ ]:
db.list_collection_names()

### 4. Working with documents

#### 4.1. Look at the data first

We use data from SPBU site about teachers of the University.

In [ ]:
!ls -la ~/__DATA/IBDT_Spring_2024/topic_2

In [ ]:
file_path = '/home/jovyan/__DATA/IBDT_Spring_2024/topic_2/teachers-20240301.json'
with open(file_path, 'r') as file:
    data = json.load(file)

In [ ]:
type(data)

In [ ]:
data[0]

In [ ]:
print('total records:', len(data))

In [ ]:
data[-1]

#### 4.2. Insert a single document

Data in MongoDB is represented (and stored) using JSON-style documents (we know it as Python dictionary). Let's look at single instance of our data first:

In [ ]:
one_doc = data[0]
one_doc

In [ ]:
type(one_doc)

In [ ]:
one_doc.keys()

In [ ]:
one_doc['Id']

In [ ]:
one_doc['Employments']

To insert a document into a collection `teachers` we can use the `insert_one()` method:

In [ ]:
result = db['teachers'].insert_one(one_doc)
result

In [ ]:
# was insert a successful?
result.acknowledged

In [ ]:
# id of a new document in a collection
result.inserted_id

We can check if documents is in our database:

In [ ]:
db.list_collection_names()

Now we see an inserted document (with use `find_one()` method):

In [ ]:
db['teachers'].find_one()

In [ ]:
db['teachers'].count_documents({})

#### 4.3. Insert many documents at once

In [ ]:
len(data)

In [ ]:
data[:3]

We can also perform bulk insert operations with use of `insert_many()`:

In [ ]:
# this will giva as an error
# guess why?
result = db['teachers'].insert_many(data)
result.inserted_ids

In [ ]:
# this will do the job
result = db['teachers'].insert_many(data[1:])
result.inserted_ids

In [ ]:
len(result.inserted_ids)

#### 4.5. Basic manipulations

Display first records in database:

In [ ]:
count = 0

# NOTE that `find()` is a generator 
# so we need to stop it somehow 

for teacher in db['teachers'].find():
    print(count)
    print(teacher)
    print('---' * 5)
    count += 1
    if count > 5: break
print('the end')

Or we can use `find(...)` to find records by condition: 

In [ ]:
for post in db['teachers'].find({'FullName': 'Гаршин Василий Владимирович'}):
    print(post)

If we just want to know how many documents match a query we can perform a `count_documents()` operation instead of a full query:

In [ ]:
db['teachers'].count_documents({})

#### 4.6. Full text search

It is not so easy... Look [here](https://www.mongodb.com/docs/manual/core/link-text-indexes/#std-label-text-search-on-premises)

In [ ]:
db['teachers'].create_index(
    [('FullName', pymongo.TEXT)], 
    name='search_index', 
    default_language='russian'
)

In [ ]:
result = db.teachers.find( { "$text": { "$search": "Гаршин" } } )

In [ ]:
list(result)

### 5. Data analysis with Mongo

Here are recommended resources to learn how to deal with data queries in Mongo:
- official MongoDB [manual](https://www.mongodb.com/developer/languages/python/python-quickstart-aggregation/)
- some [recommendations](https://developer.ibm.com/tutorials/analyze-json-data-in-mongodb-with-python/) from IBM
- nice [playground](https://mongoplayground.net/) to improve your skills and test your queries

In [ ]:
data[:3]

In [ ]:
pipeline = [
    {
        "$match": {
            "FullName": "Гаршин Василий Владимирович"
        }
    }
]
result = db.teachers.aggregate(pipeline)
list(result)

In [ ]:
# full text search

pipeline = [
    {
        "$match": { "$text": { "$search": "Гаршин" } }
    }
]
result = db.teachers.aggregate(pipeline)
list(result)

In [ ]:
pipeline = [
    {
      "$sort": {
         "FullName": pymongo.DESCENDING
      }
   }
]
result = db.teachers.aggregate(pipeline)
list(result)[:5]

In [ ]:
# limited

pipeline = [
    {
        "$sort": {
            "FullName": pymongo.DESCENDING
        }
    },
    {
        "$limit": 1
    }
]
result = db.teachers.aggregate(pipeline)
list(result)

In [ ]:
# name duplicates

list(db.teachers.aggregate([
    {
        "$group": {
            "_id": "$FullName",
            "count": {
                "$sum": 1
            }
        }
    },
    {
        "$sort": {
            "count": pymongo.DESCENDING
        }
    }
]))

In [ ]:
list(db.teachers.aggregate([
    {
        "$unwind": "$Employments"
    },
    {
        "$group": {
            "_id": "$Employments.Position",
            "count": {
                "$sum": 1
            }
        }
    }
]))

In [ ]:
# let'a add sorting and limit output

result = db.teachers.aggregate([
    {
        "$unwind": "$Employments"
    },
    {
        "$group": {
            "_id": "$Employments.Position",
            "count": {
                "$sum": 1
            }
        }
    },
    {
        "$sort": {
            "count": pymongo.DESCENDING
        }
    },
    {
        "$limit": 10
    }
])
result = list(result)
result

In [ ]:
!pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt

labels = [x['_id'] for x in result]
sizes = [x['count'] for x in result]

plt.figure(figsize=(8, 8))
plt.pie(sizes, labels=labels)
plt.show()

### 6. Relations in non-relational MongoDB

In [ ]:
one_doc = {
    'Id': 2168123, 
    'FullName': 'Гаршин Василий Владимирович', 
    'Course': "Introduction to Big Data Modern Technologies"
}

In [ ]:
db['courses'].insert_one(one_doc)

In [ ]:
db.list_collection_names()

In [ ]:
# all documents with related data

pipeline = [
    {
        "$lookup": {
            "from": "courses",       # this `database` or `collection` we want to join
            "localField": "Id",      # that will be one field to join
            "foreignField": "Id",    # and that will be the foreign field 
            "as": "related_courses", # new field with data joined
        }
    }
]
result = db.teachers.aggregate(pipeline)
list(result)

In [ ]:
pipeline = [
    {
        "$lookup": {
            "from": "courses", 
            "localField": "Id", 
            "foreignField": "Id", 
            "as": "related_courses",
        }
    },
    {
        "$match": {
        "FullName": "Гаршин Василий Владимирович"
        }
    }
]
result = db.teachers.aggregate(pipeline)
list(result)

### 7. Home assignment

In [ ]:
!ls -la ~/__DATA/IBDT_Spring_2024/topic_2

Your home assignment for this part is:
1. Take the large file `events-2022-08-29-2023-02-06-2302022348.json` (about 67 MB but it is ok)
2. Load that file as collection (or table) in database in MongoDB (let's call that database `homedb`)
3. The data contains SPBU timetable. Briefly describe the data you have loaded (key fields? how many documents are in it?). It is not required to describe data in details. __NOTE:__ the data itself is in Russian, but you do not need the knowledge of Russian to name the fields and undercover their purposes
4. Build aggregations to answer following questions below

Questions to answer with use of match and aggregation techniques:
- are there courses tha last all day? (__HINT__: look at `AllDay` field and use `match` to find desired values)
- are there courses with no teachers? (__HINT:__ look at `HasEducators` field)
- find first 5 high loaded time intervals within day (__HINT__: "high loaded" means that most of courses scheduled in that time intervals, use `TimeIntervalString` field and `$group` function)

__NOTE:__ use of JavaScript for data loading pipeline to MongoDB would be a plus